# Защита данных клиентов страховой компании.

### Введение.

Задача - защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.
Данные надо защитить так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.  

План работы:
1. Загрузка данных.  
2. Ответ на вопрос об умножении матриц.  
3. Формулирование алгоритма действий.  
4. Программирование и проверка алгоритма.
5. Вывод.

### 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.duplicated().sum()

153

Дупликаты удалять не будем, потому что не исключено, что есть разные клиенты, у которых все признаки одинаковы.  
Иных преобразований, кроме масштабирования признаков "Возраст" и "Зарплата", производить не требуется.

In [6]:
# импорт функции масштабирования
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# список признаков, которые будем масштабировать
numeric = ['Возраст','Зарплата']
# масштабирование
data[numeric]=scaler.fit_transform(data[numeric])

In [7]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,1.190432,0.978235,1,0
1,0,1.782851,-0.193589,1,1
2,0,-0.231375,-1.910918,0,0
3,0,-1.179247,0.180182,2,0
4,1,-0.349859,-1.395720,0,0


Теперь данные готовы к работе.

### 2. Умножение матриц

Ответим на вопрос, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу .

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:**   
При умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:**  
Сначала найдем связь между параметрами линейной регрессии  𝑤′ и 𝑤, где  
w' - вектор весов линейной регрессии после умножения признаков на квадратную обратимую матрицу преобразования Р.  

$$
w' = ((XP)^T XP)^{-1} (XP)^T y =
$$

$$
= (P^T X^T X P)^{-1} P^T X^T y =
$$

$$
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Т.к. произведение квадратной матрицы на матрицу, обратную ей, равно единичной матрице, то

$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$

где Е - единичная матрица.  

Если любую матрицу умножить на единичную (или наоборот), то получится эта же матрица. Поэтому

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$

Но,

$$
(X^T X)^{-1} X^T y = w
$$

Поэтому

$$
w' = P^{-1} w
$$

Т.е. параметр линейной регрессии после преобразования равен произведению матрицы, обратной матрице преобразования, на начальное значение параметра линейной регрессии.  

Предсказания после преобразования:

$$
a' = X P w' = 
$$

$$
= X P P^{-1} w =
$$

$$
= X E w =
$$

$$
= X w
$$

Т.о, получили выражение для предсказания, равное начальному:

$$
a' = X w
$$

### 3. Алгоритм преобразования

**Алгоритм**

1. Создадим обратимую матрицу со случайными значениями по нормальному распределению, размерность 4х4 - по количеству признаков за исключением целевого признака.  
2. Умножим признаки на эту матрицу.
3. Обучим модель на преобразованных признаках.

**Обоснование**

Как было показано выше, при умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

### 4. Проверка алгоритма

В качестве метрики будем использовать R2. Модель - линейная регрессия.

In [8]:
# импорт алгоритма линейной регрессии
from sklearn.linear_model import LinearRegression

In [9]:
# импорт функции расчета R2
from sklearn.metrics import r2_score

In [10]:
# выделение признаков и целевого признака
features1 = data.drop(columns='Страховые выплаты')
target = data['Страховые выплаты']

### 4.1. Качество модели без преобразования.

In [11]:
# создание и обучение модели
model1 = LinearRegression()
model1.fit(features1, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
# предсказания
pred1 = model1.predict(features1)

In [13]:
# значение метрики
r2_1=r2_score(target,pred1)
print ('R2 без преобразования:', r2_1)

R2 без преобразования: 0.4249455028666801


### 4.2. Качество модели после преобразования.

In [14]:
# создадим матрицу случайных чисел размерностью 4х4 - по ширине матрицы признаков (4).
P = np.random.normal(size=(4,4))
P

array([[ 1.15074967,  1.75257923,  0.65666769,  0.50177618],
       [-0.20777297, -0.05247044, -0.56708473, -0.36534121],
       [-1.01263854, -0.3893389 , -0.75292949, -0.18490391],
       [-0.30000527,  1.47008966,  2.39654466, -0.57254182]])

In [15]:
# проверим вспомогательную матрицу Р на обратимость - матрица обратима, если нет сообщения об ошибке
np.linalg.inv(P)

array([[-0.0985317 ,  0.74505444, -1.20081971, -0.17396754],
       [ 0.62439863,  0.38526555,  0.59848356,  0.10810219],
       [-0.29426782, -0.66487362, -0.2734316 ,  0.25466739],
       [ 0.42312374, -2.184198  ,  1.02138511, -0.31188488]])

In [16]:
# умножим признаки на вспомогательную матрицу Р
features2 = features1 @ P

In [17]:
# создание и обучение модели
model2 = LinearRegression()
model2.fit(features2, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
# предсказания
pred2 = model2.predict(features2)

In [19]:
# значение метрики
r2_2=r2_score(target,pred2)
print ('R2  c преобразованием:', r2_2)

R2  c преобразованием: 0.42494550286668


### 4.3. Сравнения качества моделей.

In [20]:
print ('R2 без преобразования:', r2_1)
print ('R2  c преобразованием:', r2_2)
print ('Разность значений R2:', r2_1-r2_2)

R2 без преобразования: 0.4249455028666801
R2  c преобразованием: 0.42494550286668
Разность значений R2: 1.1102230246251565e-16


### 5. Вывод.

Как видим, значение метрики практически не изменилось. Т.е. преобразование работает.